Author: Sofia Ingersoll


Title: WK6 SQL Q1


Date: 2024/05/14



#### Step 1

Create a query that will return the distinct species for which there is egg data (not all species and not all nests have egg data), so that you can then loop over those species. Your query should return two columns, species code and scientific name. Please order the results in alphabetic order of scientific name.


In [5]:
import pandas as pd
import duckdb

# Connect to the DuckDB database
conn = duckdb.connect(database='database.db')  # Replace 'your_database.db' with your actual database file

# Step 1: Query distinct species with egg data
species_query = """
SELECT DISTINCT Species.Code, Species.Scientific_name
FROM Bird_nests
JOIN Species ON Bird_nests.Species = Species.Code
WHERE Bird_nests.Length IS NOT NULL AND Bird_nests.Width IS NOT NULL
ORDER BY Species.Scientific_name;
"""

# Read data into Pandas DataFrame
species_df = pd.read_sql(species_query, conn) 
# should do same as above
# Execute the query and fetch the results
#species_result = conn.execute(species_query).fetchdf()
# Load results into a Pandas DataFrame
#species_df = pd.DataFrame(species_result)


# Display the DataFrame
print(species_df)


CatalogException: Catalog Error: Table with name Bird_nests does not exist!
Did you mean "temp.information_schema.tables"?
LINE 3: FROM Bird_nests
             ^

In [3]:
species_df


NameError: name 'species_df' is not defined

#### Step 2
After you’ve connected to the database and created a cursor c, iterate over the species like so:

species_query = """SELECT Code, Scientific_name FROM..."""
for row in c.execute(species_query).fetchall():  # DuckDB lame-o workaround
    species_code = row[0]
    scientific_name = row[1]
    # query egg data for that species (step 3)
    # compute statistics and print results (step 4)

In [ ]:
species_query = """
SELECT DISTINCT Species.Code, Species.Scientific_name
FROM Bird_nests
JOIN Species ON Bird_nests.Species = Species.Code
WHERE Bird_nests.Length IS NOT NULL AND Bird_nests.Width IS NOT NULL
ORDER BY Species.Scientific_name;
"""
for row in conn.execute(species_query).fetchall():  # DuckDB lame-o workaround
    species_code = row[0]
    scientific_name = row[1]
    # query egg data for that species (step 3)
    # compute statistics and print results (step 4)

#### Step 3

You will need to construct a query that gathers egg data for a given species, one species at a time; the species code will be a parameter to that query. You can compute the formula

in SQL or in Pandas. For simplicity, SQL is suggested:

```{SQL}
egg_query = """SELECT Width*Width*Length AS Volume FROM..."""
```

Within the loop, you will want to execute the query on the current species in the loop iteration. You may use the Pandas function pd.read_sql to do so and also directly load the results into a dataframe:
```{python}
df = pd.read_sql(egg_query, conn, ...)
```

Do a `help(pd.read_sql)` to figure out how to pass parameters to a query.

You may get a bunch of warnings from Pandas about how it “only supports SQLAlchemy…”. Just ignore them. (Sorry about that.)


In [ ]:
species_query = """
SELECT DISTINCT Species.Code, Species.Scientific_name
FROM Bird_nests
JOIN Species ON Bird_nests.Species = Species.Code
WHERE Bird_nests.Length IS NOT NULL AND Bird_nests.Width IS NOT NULL
ORDER BY Species.Scientific_name;
"""

# Read data into Pandas DataFrame
species_df = pd.read_sql(species_query, conn)

# Step 2: Iterate over each species code and execute the egg query
for index, row in species_df.iterrows():
    species_code = row['Code']
    egg_query = f"""
    CREATE TEMP TABLE EggVolumes AS
    SELECT AVG(Width * Width * Length) AS EggVolume
    FROM Bird_nests
    JOIN Species ON Bird_nests.Species = Species.Code
    WHERE Species = '{species_code}' AND Length IS NOT NULL AND Width IS NOT NULL
    GROUP BY Species.Code
    """
    egg_volume_df = pd.read_sql(egg_query, conn)
    # Do something with the egg_volume_df, like printing it
    print(egg_volume_df)


In [ ]:
# Query to iterate through all species and collects the egg data for each species at a time using species code 
species_query = """
SELECT DISTINCT Species.Code, Species.Scientific_name
FROM Bird_nests
JOIN Species ON Bird_nests.Species = Species.Code
WHERE Bird_nests.Length IS NOT NULL AND Bird_nests.Width IS NOT NULL
ORDER BY Species.Scientific_name;
"""

# Get a list of all distinct species codes
# Iterate over each species code
egg_query = """CREATE TEMP TABLE EggVolumes AS
SELECT Species.Code,
       AVG(Width * Width * Length) AS EggVolume
FROM Bird_nests
JOIN Species ON Bird_nests.Species = Species.Code
WHERE Bird_nests.Length IS NOT NULL AND Bird_nests.Width IS NOT NULL
GROUP BY Species.Code;
"""


for row in conn.execute(species_query).fetchall():  # DuckDB lame-o workaround
    species_code = row['Code']
    scientific_name = row[1]


#### Step 4

Finally, and still within your loop, you’ll want to compute statistics and print out the results:

cv = round(df.Volume.std()/df.Volume.mean()*100, 2)
print(f"{scientific_name} {cv}%")

Your output should look like this:

Arenaria interpres 21.12%
Calidris alpina 5.46%
Calidris fuscicollis 16.77%
Charadrius semipalmatus 8.99%
Phalaropus fulicarius 4.65%
Pluvialis dominica 19.88%
Pluvialis squatarola 6.94%



In [ ]:
# Step 2: Iterate over each species code and execute the egg query
for index, row in species_df.iterrows():
    species_code = row['Code']
    egg_query = f"""
    CREATE TEMP TABLE EggVolumes AS
    SELECT AVG(Width * Width * Length) AS EggVolume
    FROM Bird_nests
    JOIN Species ON Bird_nests.Species = Species.Code
    WHERE Species = '{species_code}' AND Length IS NOT NULL AND Width IS NOT NULL
    GROUP BY Species.Code
    """
    egg_volume_df = pd.read_sql(egg_query, conn)
    # Do something with the egg_volume_df, like printing it
   # Compute statistics and print out the results
    if not egg_volume_df.empty:
        cv = round(egg_volume_df['EggVolume'].std() / egg_volume_df['EggVolume'].mean() * 100, 2)
        print(f"{scientific_name} {cv}%")
